In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.integrate import solve_ivp 

In [ ]:
def f(z,V,k,m,a,α):                                     # def f(r,V,k,m,a,c,α):
    Vt = 0.0;
    R = 1.0;
    c = 1.0;
    ################################################
    # Γt
    G = (1-4*m**2+3*z**2+4*a**2 * m**2 -8*m**2 * z**2 + 3*z**4 - 4*m**2 * z**4 + z**6)
    ###
    Γt_tr = (m*(z+2*z**3+z**5+2*m-a*m*a**2+4*m*z**2+4*m*a**2 * z**2 + 2*m*z**4))/ (z**2 * G)
    Γt_tz = (m*z*(z+2*z**3+z**5+2*m-6*m*a**2+4*m*z**2+2*m*z**4))/(z**2 * G)
    Γt_rt = Γt_tr
    Γt_rϕ = (-m*a*(z+4*m))/(G)
    Γt_ϕr = Γt_rϕ
    Γt_ϕz = (-m*a*z*(3*z+4*m))/(G)
    Γt_zt = Γt_tz
    Γt_zϕ = Γt_ϕz
    ################################################
    # Γr
    H = (z+2*m)*z**2
    ####
    Γr_tt = m/H
    Γr_tϕ = (2*m*a)/H
    Γr_rr = -m/H
    Γr_rz = -m*z/H
    Γr_ϕt = 2*m*a/H
    Γr_ϕϕ = -1*(z**3+m+2*m*z**2)/H
    Γr_zr = (-m*z)/H
    Γr_zz = m/H
#######################
    # Γϕ
    Γϕ_tz = ((a*m*z)*(-4*m+3*z))/(G)
    Γϕ_zϕ = (-1*(m*z)*(z+2*z**3+z**5-2*m+6*m*a**2-4*m*z**2-2*m*z**4))/((1+z**2)*G)
    Γϕ_ϕz = -Γϕ_zϕ    
#######################
    # Γz
    L = (1+z**2)*(z+2*m)
    Γz_tt = (m*z)/(L)
    Γz_tϕ = (-1*(3*m*a*z)/(L*(1+z**2)))
    Γz_rr = (m*z)/(L)
    Γz_rz = (-m*R)/(L)
    Γz_ϕt = (-3*m*a*z*R**2)/(L*(1+z**2))
    Γz_ϕϕ = (m*z)/(L*(1+z**2))
    Γz_zr = -m/L
    Γz_zz = (-m*z)/((1+z**2)*(np.sqrt(z)+2*m))
#######################################
    Vz,Vϕ,Bϕ = V
    γ = 1./(np.sqrt(1-(Vz**2 + Vϕ**2)))
    Uϕ = γ*Vϕ;
    U0 = γ;
    Uz = γ*Vz;  
    gzz = -z/(z+2*m)
    Bz = 1e3
    ###############################
    ## dBϕdz
    L1 = ((z**2/(4*np.pi*(z+2*m)**2))*((1./(U0*Uz)) + ((2*(Bϕ*Uz-Bz*Uϕ))/U0) \
                                           + ((z**3 * Bz)/(2*m*a*U0**2))))
    
    
    
    L3 = (-2*Vz*Vϕ*Γt_ϕz) + (-1/Vz) \
    * (Γz_tt+2*Vϕ*Γz_tϕ-2*Γt_tz*Vz**2+Γz_ϕϕ*Vϕ**2 + Vz**2 * (Γz_zz-2*Vϕ*Γt_ϕz)) \
    + gzz*(2.5*k*z**-3.5) + (Bϕ/4*np.pi)* (((2*z**2+4*z*m-4*z)/((z+2*m)**3)) \
       + (((z**2)/((z+2*m)**2))*(Γϕ_ϕz+Γr_zr+Γz_zz +Γt_tz)))\
    *(((z*Bϕ)/(z+2*m)) - ((Vz*Bz*z**3)/(2*m*a)))
    
    
    L4 = ((Bϕ*(2*z**2 + 4*m*z-4*z))/(4*np.pi*(z+2*m)**3))\
    *2*((((Bϕ*Uz - Bz*Uϕ))/(U0)) + ((Bz*z**3)/(2*m*a*U0**2)))
    
    L5 = (2*Ur/U0)*(((Bz**2 * Γϕ_tr * z**4)/(4*np.pi * 2*m*a*(z+2*m))) \
                     + ((z**2 * (Bz**2 + Bϕ**2))/(4*np.pi * (z+2*m)**2))\
                     *(Γt_tr + Γr_rr + Γϕ_ϕr + Γz_rz))
    
    L2 = Vz*(-1.5*z**(-2.5) - -2.5*k*z**(-3.5)) + (z**-1.5 + k*z**-2.5)*L3 - (L4+L5)
    
    dBϕdz = (L1**-1)*(L2)
######################################################################   
# dVzdz
    M2 = ((z**2)/(4*np.pi * (z+2*m)**2 )) \
    * (((Bϕ*(2*z+4*m-4))/(z*(z+2*m))) + dBϕdz \
       + Bϕ*(Γϕ_ϕz + Γr_zr + Γz_zz + Γt_tz))
    
    M3 = ((Bϕ*z)/(z+2*m)) - ((Vz*z**3 * Bz)/(2*m*a))
    M4 = Γz_tt+Vz**2 * (Γz_zz - 2*Vϕ*Γt_ϕz-2*Γt_tz)\
    +Vϕ*(2*Γz_tϕ+Vϕ*Γz_ϕϕ)
    
    M1 = ((z**-1.5+k*z**-2.5)*U0**2)*(M4) + gzz*(2.5*k*z**-3.5) + M2*M3
    
    dVzdz = ((-1./(Vz*U0**2*(ρ+P)))**-1)*(M1)
   ##################################################################3 
    # dVϕdz
    N2 = (2*(z**-1.5 + k*z**-2.5)*U0**2 * Vz)\
    *(Γϕ_tz - Vϕ * Γt_tz - Vϕ**2 * Γt_zϕ )
    
    N3 = ((z**2)/((z+2*m)**2 ))*(((Bϕ*(2*z+4*m-4))/(z*(z+2*m))) + dBϕdz \
       + Bϕ*(Γϕ_ϕz + Γr_zr + Γz_zz + Γt_tz))
    
    N4 = (Bz/4*np.pi)*(((Vϕ*z**3)/(2*m*a)) + (z/(z+2*m)))
    N5 = z/(z+2*m) - (z**3/(2*m*a)) + Uϕ*(Uϕ+Uz)
    N6 = (α*k*z**-2.5)*(2*Γϕ_ϕr + Γr_zr)
    N1 = N2 - N3*N4 + N5*N6
    dVϕdz = (((z**-1.5 * k*z**-2.5)*Vz*U0**2)**-1)*(N1)
    
    dVdz = [dVzdz, dVϕdz,Bϕ]
    return dVdz